In [1]:
import os
import sys
# 如果当前代码文件运行测试需要加入修改路径，避免出现后导包问题
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, os.path.join(BASE_DIR))

PYSPARK_PYTHON = "/miniconda2/envs/reco_sys/bin/python"
# 当存在多个版本时，不指定很可能会导致出错
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_PYTHON

from offline import SparkSessionBase
import pyhdfs
import time

In [2]:
class UpdateUserProfile(SparkSessionBase):
    """离线相关处理程序
    """
    SPARK_APP_NAME = "updateUser"
    ENABLE_HIVE_SUPPORT = True

    #SPARK_EXECUTOR_MEMORY = "7g"

    def __init__(self):

        self.spark = self._create_spark_session()

uup = UpdateUserProfile()

In [3]:
import pandas as pd
from datetime import datetime

In [4]:
def datelist(beginDate, endDate):
    date_list=[datetime.strftime(x,'%Y-%m-%d') for x in list(pd.date_range(start=beginDate, end=endDate))]
    return date_list


In [5]:
dl = datelist("2019-03-05", time.strftime("%Y-%m-%d", time.localtime()))


In [6]:
dl

['2019-03-05',
 '2019-03-06',
 '2019-03-07',
 '2019-03-08',
 '2019-03-09',
 '2019-03-10',
 '2019-03-11',
 '2019-03-12',
 '2019-03-13',
 '2019-03-14',
 '2019-03-15',
 '2019-03-16',
 '2019-03-17',
 '2019-03-18',
 '2019-03-19',
 '2019-03-20',
 '2019-03-21',
 '2019-03-22',
 '2019-03-23',
 '2019-03-24',
 '2019-03-25',
 '2019-03-26',
 '2019-03-27',
 '2019-03-28',
 '2019-03-29',
 '2019-03-30',
 '2019-03-31',
 '2019-04-01',
 '2019-04-02',
 '2019-04-03',
 '2019-04-04',
 '2019-04-05',
 '2019-04-06',
 '2019-04-07',
 '2019-04-08',
 '2019-04-09',
 '2019-04-10',
 '2019-04-11',
 '2019-04-12',
 '2019-04-13',
 '2019-04-14',
 '2019-04-15',
 '2019-04-16',
 '2019-04-17',
 '2019-04-18',
 '2019-04-19',
 '2019-04-20',
 '2019-04-21',
 '2019-04-22',
 '2019-04-23',
 '2019-04-24',
 '2019-04-25',
 '2019-04-26',
 '2019-04-27',
 '2019-04-28',
 '2019-04-29',
 '2019-04-30',
 '2019-05-01',
 '2019-05-02',
 '2019-05-03',
 '2019-05-04',
 '2019-05-05',
 '2019-05-06',
 '2019-05-07',
 '2019-05-08',
 '2019-05-09',
 '2019-05-

In [ ]:
fs = pyhdfs.HdfsClient(hosts='hadoop-master:50070')
for d in dl:
    try:
        _localions = '/user/hive/warehouse/profile.db/user_action/' + d
        if fs.exists(_localions):
            uup.spark.sql("alter table user_action add partition (dt='%s') location '%s'" % (d, _localions))
    except Exception as e:
        # 已经关联过的异常忽略,partition与hdfs文件不直接关联
        pass

In [8]:
uup.spark.sql('use profile')


DataFrame[]

In [9]:
sqlDF = uup.spark.sql(
"select actionTime, readTime, channelId, param.articleId, param.algorithmCombine, param.action, param.userId from user_action where dt>='2019-04-01'")


In [10]:
sqlDF.show()

+-------------------+--------+---------+--------------------+----------------+--------+-------------------+
|         actionTime|readTime|channelId|           articleId|algorithmCombine|  action|             userId|
+-------------------+--------+---------+--------------------+----------------+--------+-------------------+
|2019-04-04 08:31:44|        |        0|[14805, 15196, 44...|              C2|exposure|1113004557979353088|
|2019-04-04 08:31:44|        |        0|[14805, 15196, 44...|              C2|exposure|1113004557979353088|
|2019-04-04 08:31:44|        |        0|[14805, 15196, 44...|              C2|exposure|1113004557979353088|
|2019-04-04 08:31:44|        |        0|[14805, 15196, 44...|              C2|exposure|1113004557979353088|
|2019-04-04 08:31:44|        |        0|[14805, 15196, 44...|              C2|exposure|1113004557979353088|
|2019-04-04 08:31:44|        |        0|[14805, 15196, 44...|              C2|exposure|1113004557979353088|
|2019-04-04 08:31:44|       

In [ ]:
if sqlDF.collect():
    def _compute(row):
        # 进行判断行为类型
        _list = []
        if row.action == "exposure":
            for article_id in eval(row.articleId):
                _list.append(
                    [row.userId, row.actionTime, article_id, row.channelId, False, False, False, True, row.readTime])
            return _list
        else:
            class Temp(object):
                shared = False
                clicked = False
                collected = False
                read_time = ""

            _tp = Temp()
            if row.action == "share":
                _tp.shared = True
            elif row.action == "click":
                _tp.clicked = True
            elif row.action == "collect":
                _tp.collected = True
            elif row.action == "read":
                _tp.clicked = True
            else:
                pass
            _list.append(
                [row.userId, row.actionTime, int(row.articleId), row.channelId, _tp.shared, _tp.clicked, _tp.collected,
                 True,
                 row.readTime])
            return _list
    # 进行处理
    # 查询内容，将原始日志表数据进行处理
    _res = sqlDF.rdd.flatMap(_compute)
    data = _res.toDF(["user_id", "action_time","article_id", "channel_id", "shared", "clicked", "collected", "exposure", "read_time"])

In [11]:
# 获取基本用户行为信息，然后进行文章画像的主题词合并
uup.spark.sql("use profile")
# 取出日志中的channel_id
user_article_ = uup.spark.sql("select * from user_article_basic").drop('channel_id')
uup.spark.sql('use article')
article_label = uup.spark.sql("select article_id, channel_id, topics from article_profile")
# 合并使用文章中正确的channel_id
click_article_res = user_article_.join(article_label, how='left', on=['article_id'])

In [12]:
click_article_res.show()

+-------------------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------------------+
|         article_id|            user_id|        action_time|shared|clicked|collected|exposure|read_time|channel_id|              topics|
+-------------------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------------------+
|              13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|[补码, 字符串, 李白, typ...|
|              13401|1106396183141548032|2019-03-28 10:58:20| false|  false|    false|    true|         |        18|[补码, 字符串, 李白, typ...|
|              14805|1105045287866466304|2019-03-11 18:15:48| false|  false|    false|    true|         |        18|[占位符, Code, sep, ...|
|              14805|                  1|2019-03-05 17:34:03| false|  false|    false|    true|         |        18|[占位符, Code, sep, ...|
|              14805|1111524501104

In [13]:
import pyspark.sql.functions as F
click_article_res = click_article_res.withColumn('topic', F.explode('topics')).drop('topics')

In [15]:
click_article_res.show(100)

+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------+
|article_id|            user_id|        action_time|shared|clicked|collected|exposure|read_time|channel_id|   topic|
+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------+
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|      补码|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|     字符串|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|      李白|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|    type|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|      元素|
|     13401|                 10|2019-03-06 10:06:12| false|  fal

In [22]:
def compute_weights(row):
    """处理每个用户对文章的点击数据
    """
    weightsOfaction = {
        "read_min": 1,
        "read_middle": 2,
        "collect": 2,
        "share": 3,
        "click": 5
    }

    
    from datetime import datetime
    import numpy as np


    # 读取文章的标签数据
    # 计算权重值
    # 时间间隔


    t = datetime.now() - datetime.strptime(row.action_time, '%Y-%m-%d %H:%M:%S')
    # 时间衰减系数
    time_exp = 1 / (np.log(t.days + 1) + 1)

    if row.read_time == '':
        r_t = 0
    else:
        r_t = int(row.read_time)
    # 浏览时间分数
    is_read = weightsOfaction['read_middle'] if r_t > 1000 else weightsOfaction['read_min']

    # 每个词的权重分数
    weigths = time_exp * (
                row.shared * weightsOfaction['share'] + row.collected * weightsOfaction['collect'] + row.
                clicked * weightsOfaction['click'] + is_read)

    return row.article_id, row.user_id, row.channel_id, row.topic, float(weigths)

In [23]:
click_article_result = click_article_res.rdd.map(compute_weights).toDF(['article_id', 'user_id', 'channel_id', 'topic', 'weights'])

In [25]:
click_article_result.registerTempTable('weightstable')

In [26]:
click_article_result.show()

+----------+-------------------+----------+--------+-------------------+
|article_id|            user_id|channel_id|   topic|            weights|
+----------+-------------------+----------+--------+-------------------+
|     13401|                 10|        18|      补码| 0.1414742204633455|
|     13401|                 10|        18|     字符串| 0.1414742204633455|
|     13401|                 10|        18|      李白| 0.1414742204633455|
|     13401|                 10|        18|    type| 0.1414742204633455|
|     13401|                 10|        18|      元素| 0.1414742204633455|
|     13401|                 10|        18|    删除元素| 0.1414742204633455|
|     13401|                 10|        18|      负数| 0.1414742204633455|
|     13401|                 10|        18|      基数| 0.1414742204633455|
|     13401|                 10|        18|     tp2| 0.1414742204633455|
|     13401|                 10|        18|    数据类型| 0.1414742204633455|
|     13401|                 10|        18|     二进制

In [42]:
def add_weight(row):
    w = 0
    for item in row.weights:
        w += item
        
    return row.user_id, row.channel_id, row.topic[0], w

In [43]:
all_weights = uup.spark.sql('select user_id, max(channel_id) channel_id, collect_set(topic) topic, collect_list(weights) weights from weightstable group by user_id, topic').rdd.map(add_weight).toDF(['user_id', 'channel_id', 'topic', 'weights'])

In [44]:
all_weights.sort('user_id').show()

+-------+----------+---------+-------------------+
|user_id|channel_id|    topic|            weights|
+-------+----------+---------+-------------------+
|      1|        18|   people| 0.9980445233748928|
|      1|        18|    wusir| 0.9906443420818408|
|      1|        18|     INFO| 0.1418968623472544|
|      1|        13|    ldquo| 0.9949567868432247|
|      1|        18|      mod| 0.1418968623472544|
|      1|        18| deepcopy|0.14175470946941152|
|      1|        18|   wechat|0.14175470946941152|
|      1|        18|publisher|0.14175470946941152|
|      1|        18|   logger| 0.2833248205700103|
|      1|        13|       人类| 0.9949567868432247|
|      1|        18|QueryDict| 0.1414279582227559|
|      1|        18|       异构|0.14175470946941152|
|      1|        18|   代码自动补全| 0.1414742204633455|
|      1|        18|     form| 0.2828559164455118|
|      1|        18|    money| 0.2837937246945088|
|      1|        18|    price|0.28350941893882303|
|      1|        18|      MD5| 

In [ ]:
def save_to_hbase(partition):
    #  用于读取hbase缓存结果配置
    import happybase
    pool = happybase.ConnectionPool(size=10, host='192.168.19.137', port=9090)
#        with pool.connection() as conn:
#            table = conn.table('user_profile')
#            table.put('user:{}'.format(row.user_id).encode(),
#                      {'partial:{}:{}'.format(row.channel_id, row.topic).encode(): json.dumps(
#                          weigths).encode()})
#            conn.close()

all_weights.foreachPartition(save_to_hbase)